In [ ]:
# Descrição
# Este notebook conta o número de registros de cada arquivo em cada etapa de processamento.
#
# Description
# This notebook counts the number of register in each file in each processing step.

In [ ]:
# Spark Config
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# spark = sparkSession
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

# installing necessary packages for notebook session
sc.install_pypi_package("boto3")

In [ ]:
from dateutil import parser
# lib boto3 to use aws services
import boto3

# counting file sizes aka number of records
counters = {"MO_1510" + str(day): {"raw_data": 0, "hour-6-23": 0, "hour-6-23-in-sp": 0} for day in range(1,32)}


for key in counters.keys():
    # reading raw data
    traces_raw = spark.read.csv("s3a://mobility-traces-sp/raw-data/" + key + ".csv")
    # count raw data records
    counters[key]["raw_data"] = traces_raw.count()
    
    # reading records_between_6-23_hour
    traces_hour_6_23 = spark.read.parquet("s3a://mobility-traces-sp/processed-data/using-server-hour/records-between-6-23-server-hour/" + key + "/")
    # counting records
    counters[key]["hour-6-23"] = traces_hour_6_23.count()
    
    # reading records_between_6_23_only_in_sp
    traces_hour_6_23_in_sp = spark.read.parquet("s3a://mobility-traces-sp/processed-data/using-server-hour/records-between-6-23-only-sp-server-hour/" + key + "/")
    # counting records
    counters[key]["hour-6-23-in-sp"] = traces_hour_6_23_in_sp.count()

# file header
csv_out = "day,raw_data_count,hour_6_23_count,hour_6_23_in_sp_count\n"
for key in counters.keys():
    # writing csv text
    csv_out += key +","+str(counters[key]["raw_data"])+","+str(counters[key]["hour-6-23"])+","+str(counters[key]["hour-6-23-in-sp"])+"\n"
    
s3 = boto3.client('s3')

# writing results in S3
s3.put_object(Body=bytes(csv_out,"utf-8"), Bucket='mobility-traces-sp', Key='statistics/number-of-records/number-records-per-file-using-server-hour.csv')

In [ ]:
# Results